In [19]:
import sys
import os
import urllib3
import json
import numpy as np
import datetime
import time
import random
import pickle

In [20]:
grid_size = 25

In [21]:
#generate lattice of points to sample within the state
border = []
max_lat = -90.0 #set to entire globe
min_lat = 90.0
max_lon = -180.0
min_lon = 180.0
with open("/home/chase/projects/peakload/data/stateborders/texasborder.txt", 'r') as d:
    lines = d.readlines()
    for line in lines:
        tokens = line.strip().split(",")
        lon = float(tokens[0])
        lat = float(tokens[1])
        border.append((lat, lon))
        if lat > max_lat:
            max_lat = lat
        if lat < min_lat:
            min_lat = lat
        if lon > max_lon:
            max_lon = lon
        if lon < min_lon:
            min_lon = lon
            
bounding_box = [max_lat, min_lat, max_lon, min_lon]

row_len = int(np.sqrt(grid_size))
lambdas = np.arange(0,1+(1/float(row_len)),1/float(row_len))
grid = np.zeros((row_len+1,row_len+1,2))
row_starts = [ (1 - i)*min_lon + i*max_lon for i in lambdas ]
col_starts = [ (1 - i)*min_lat + i*max_lat for i in lambdas ]


In [22]:
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        grid[i,j,0] = row_starts[i]
        grid[i,j,1] = col_starts[j] #reversing to read from north to south

In [ ]:
gmap = gmplot.GoogleMapPlotter(31.9686, -99.9018, 3)

lats = []
lons = []
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        lats.append(grid[i,j,1])
        lons.append(grid[i,j,0])

#gmap.scatter(lats, lons, '#3B0B39', size=40, marker=True)
for i in range(len(lats)):
    gmap.marker(lats[i], lons[i])

gmap.draw("/home/chase/projects/peakload/figs/texas_grid25.html")

In [51]:
startdate = datetime.datetime(year=2010, month=1, day=1, hour=0, minute=0, second=0)
curr = startdate
#enddate = datetime.datetime(year=2011, month=1, day=1, hour=0, minute=0, second=0)
enddate = datetime.datetime(year=2018, month=3, day=31, hour=23, minute=0, second=0)

alldays = []

while curr < enddate:
    cunix = int(time.mktime(curr.timetuple()))
    alldays.append(cunix)
    curr += datetime.timedelta(days=1)


In [33]:
36 * 365

13140

In [ ]:
#data request files

In [28]:
#want to make files with batch requests, per day with all grid point
basepath = "/home/chase/projects/peakload/data/weather/ercot/"
reqpath = basepath + "requests/"
with open("/home/chase/api_keys/darksky_key.txt", 'r') as d:
    pkey = d.readline().strip()
baseurl = "https://api.darksky.net/forecast/" + "/"
http = urllib3.PoolManager()

In [29]:


all_reqs = []
for day in alldays:
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            lat = grid[i,j,0]
            lon = grid[i,j,1]
            requrl = baseurl + str(lon) + "," + str(lat) +  "," + str(day) + "?exclude=flags"
            all_reqs.append(requrl)
            
with open(reqpath + "/grid" + str(grid_size) + "_all.txt", 'w') as d:
    for r in all_reqs:
        d.write(r + "\n")

In [52]:
#need to do this on the desktop

with open(reqpath + "grid25_all.txt", 'r') as d:
    lines = d.readlines()
    for line in lines:
        cureq = d.readline().strip()
        tokens = cureq.split("/")
        tokens = tokens[-1].split(",")
        lat = tokens[1]
        lon = tokens[0]
        posix = int(tokens[2].split("?")[0])
        year = str(datetime.datetime.fromtimestamp(posix).year)
        outpath = basepath + "/grid" + str(grid_size) + "/" + year + "/" + str(posix)
        if not os.path.exists(outpath):
            os.mkdir(outpath)
        response = http.request('GET', cureq)
        with open(outpath + "/grid_" + str(lat) + "," + str(lon) + ".json", 'w') as f:
            payload = json.loads(response.data)
            out = json.dumps(payload)
            f.write(out)
        time.sleep(1)
        print(posix)
    

IndexError: list index out of range

In [36]:
#request weather by cities
cities = {"Witchita Falls": (33.9137, -98.4934), 
          "Mineral Wells": (32.8085, -98.1128), 
          "Paris": (33.6609, -95.5555), 
          "Dallas": (32.7767, -96.7970), 
          "Fort Worth": (32.7555, -97.3308), 
          "Tyler": (32.3513, -95.3011),  
          "Waco": (31.5493, -97.1467), 
          "San Angelo": (31.4638, -100.4370), 
          "Abilene": (32.4487, -99.7331), 
          "Midland": (31.9973, -102.0779), 
          "Wink": (31.7512, -103.1599), 
          "Junction": (30.4894, -99.7720), 
          "Austin": (30.2672, -97.7431), 
          "Houston": (29.7604, -95.3698), 
          "Galveston": (29.3013, -94.7977), 
          "Victoria": (28.8053, -97.0036), 
          "Corpus Christi": (27.8006, -97.39640), 
          "Laredo": (27.5306, -99.4803),
          "San Antonio": (29.4241, -98.4936)}

In [53]:
all_reqs = []
for day in alldays:
    for city in cities.keys():
        lat = cities[city][0]
        lon = cities[city][1]
        requrl = baseurl + str(lat) + "," + str(lon) +  "," + str(day) + "?exclude=flags"
        all_reqs.append(requrl)
            
with open(reqpath + "/major_cities.txt", 'w') as d:
    for r in all_reqs:
        d.write(r + "\n")

In [85]:
with open(reqpath + "major_cities.txt", 'r') as d:
    lines = d.readlines()
    for line in lines:
        cureq = d.readline().strip()
        tokens = cureq.split("/")
        tokens = tokens[-1].split(",")
        lat = tokens[1]
        lon = tokens[0]
        posix = int(tokens[2].split("?")[0])
        year = str(datetime.datetime.fromtimestamp(posix).year)
        outpath = basepath + "/cities/" + year + "/" + str(posix)
        if not os.path.exists(outpath):
            os.mkdir(outpath)
        response = http.request('GET', cureq)
        with open(outpath + "/grid_" + str(lat) + "," + str(lon) + ".json", 'w') as f:
            payload = json.loads(response.data)
            out = json.dumps(payload)
            f.write(out)
        time.sleep(1)
        print(posix)

IndexError: list index out of range

In [ ]:
#process weather features
#need a vector for each hour of 2011-2017 across all latitude-longitudes, one big array for each year

In [38]:
startdate = datetime.datetime(year=2010, month=1, day=1, hour=0, minute=0, second=0)
curr = startdate
enddate = datetime.datetime(year=2018, month=1, day=1, hour=0, minute=0, second=0)

alldays = []

while curr < enddate:
    cunix = int(time.mktime(curr.timetuple()))
    alldays.append(cunix)
    curr += datetime.timedelta(days=1)

In [39]:
startdate = datetime.datetime(year=2010, month=1, day=1, hour=0, minute=0, second=0)
curr = startdate
enddate = datetime.datetime(year=2018, month=1, day=1, hour=0, minute=0, second=0)

allhours = []

while curr < enddate:
    cunix = int(time.mktime(curr.timetuple()))
    allhours.append(cunix)
    curr += datetime.timedelta(hours=1)

In [47]:
#what features are available to me?
#weatherdir = "/home/chase/projects/peakload/data/weather/ercot/grid25/"
weatherdir = "/home/chase/projects/peakload/data/weather/ercot/major_cities/"

outdir = weatherdir + "array_formatted/"
years = range(2010,2018)

In [48]:
tkeys = ['currently', 'daily', 'hourly', 'latitude', 'longitude']

#files names are grid_longitude,latitude.json
def parse_filename(fnamestr):
    token = fnamestr.split("_")[1]
    token_ = token[:-5] #break off .json extension
    tokens = token_.split(",")
    #return (lat, long)
    return((float(tokens[1]), float(tokens[0])))

In [49]:
#want to scan ahead to get all keys and use map for missing data points
#dont' need k + i + lk, just want lk's for each hour
lkeys = {'daily': set(), 'hourly': set()}

grid_locs = set()

for year in years:
    days = os.listdir(weatherdir + str(year))
    for day in days:
        wms = os.listdir(weatherdir + str(year) + "/" + day)
        for fname in wms:
            loc = parse_filename(fname)
            sloc = str(loc[0]) + "," + str(loc[1])
            grid_locs.add(sloc)
            with open(weatherdir + str(year) + "/" + day + "/" + fname, 'r') as d:
                data = json.load(d)
            for k in tkeys:
                if k == 'daily' or k == 'hourly':
                    curr = data[k]['data'] #list of dicts
                    for di in range(len(curr)):
                        d = curr[di]
                        for lk in d:
                            lkeys[k].add(lk)
                #elif k == 'currently':
                #    curr = data[k]
                #    for lk in curr.keys():
                #        lkeys[k].add(lk)

KeyError: 'daily'

In [50]:
data

{'code': 400, 'error': 'The given location is invalid.'}

In [54]:
grid_locs = sorted(list(grid_locs))
with open(outdir + "grid_locs.pck", 'wb') as f:
    pickle.dump(grid_locs, f)

In [55]:
with open(outdir + "grid_locs.pck", 'rb') as f:
    grid_locs = pickle.load(f)
grid_locs = sorted(list(grid_locs))

In [57]:
#for each hour, create a vector with an element for each key in lkeys
#assign an index for each element in lkeys

lkeymap = {}
i = 0
for tk in lkeys:
    lkeymap[tk] = {}
    for lk in lkeys[tk]:
        lkeymap[tk][lk] = i
        i += 1

lkeymax = i - 1

print(lkeymap)

{'daily': {'moonPhase': 0, 'apparentTemperatureLowTime': 1, 'apparentTemperatureMinTime': 2, 'pressure': 3, 'summary': 4, 'sunsetTime': 5, 'precipIntensity': 6, 'apparentTemperatureHighTime': 7, 'temperatureLowTime': 8, 'time': 9, 'temperatureHigh': 10, 'apparentTemperatureMax': 11, 'windBearing': 12, 'temperatureHighTime': 13, 'temperatureMax': 14, 'dewPoint': 15, 'precipAccumulation': 16, 'precipProbability': 17, 'windSpeed': 18, 'precipIntensityMaxTime': 19, 'precipIntensityMax': 20, 'icon': 21, 'cloudCover': 22, 'temperatureMin': 23, 'apparentTemperatureHigh': 24, 'apparentTemperatureLow': 25, 'temperatureMinTime': 26, 'temperatureMaxTime': 27, 'precipType': 28, 'sunriseTime': 29, 'visibility': 30, 'apparentTemperatureMaxTime': 31, 'apparentTemperatureMin': 32, 'temperatureLow': 33, 'humidity': 34}, 'hourly': {'temperature': 35, 'windSpeed': 36, 'windBearing': 37, 'pressure': 38, 'summary': 39, 'precipType': 40, 'dewPoint': 41, 'icon': 42, 'precipIntensity': 43, 'visibility': 44, '

In [58]:
lkeymax

50

In [59]:
#save lkeymap to 
with open(outdir + "data_keys.pck", 'wb') as d:
    pickle.dump(lkeymap, d)


In [6]:
#load lkeymap
with open(outdir + "data_keys.pck", 'rb') as d:
    lkeymap = pickle.load(d)

In [7]:
lkeymap

{'daily': {'apparentTemperatureHigh': 24,
  'apparentTemperatureHighTime': 7,
  'apparentTemperatureLow': 25,
  'apparentTemperatureLowTime': 1,
  'apparentTemperatureMax': 11,
  'apparentTemperatureMaxTime': 31,
  'apparentTemperatureMin': 32,
  'apparentTemperatureMinTime': 2,
  'cloudCover': 22,
  'dewPoint': 15,
  'humidity': 34,
  'icon': 21,
  'moonPhase': 0,
  'precipAccumulation': 16,
  'precipIntensity': 6,
  'precipIntensityMax': 20,
  'precipIntensityMaxTime': 19,
  'precipProbability': 17,
  'precipType': 28,
  'pressure': 3,
  'summary': 4,
  'sunriseTime': 29,
  'sunsetTime': 5,
  'temperatureHigh': 10,
  'temperatureHighTime': 13,
  'temperatureLow': 33,
  'temperatureLowTime': 8,
  'temperatureMax': 14,
  'temperatureMaxTime': 27,
  'temperatureMin': 23,
  'temperatureMinTime': 26,
  'time': 9,
  'visibility': 30,
  'windBearing': 12,
  'windSpeed': 18},
 'hourly': {'apparentTemperature': 47,
  'cloudCover': 49,
  'dewPoint': 41,
  'humidity': 48,
  'icon': 42,
  'preci

In [18]:
#all_days_hourly_arrays = {}

#the data is being delivered to me in PST

for year in years:
    days = os.listdir(weatherdir + str(year))
    for day in days:
        wms = os.listdir(weatherdir + str(year) + "/" + day)
        for fname in wms:
            loc = parse_filename(fname)
            sloc = str(loc[0]) + "," + str(loc[1])
            loc_i = grid_locs.index(sloc)
            with open(weatherdir + str(year) + "/" + day + "/" + fname, 'r') as d:
                data = json.load(d)

            for k in tkeys:
                if k == 'hourly':
                    curr = data[k]['data'] #list of dicts
                    
                    #need to just pull out time stamps from data and start/stop accordingly
                    for i in range(len(curr)):
                        time_int = curr[i]['time']
                        for key in curr[i]:
                            if time_int not in hourly_data.keys():
                                print(time_int)
                                pass
                            else:
                                hourly_data[time_int][sloc][key] = curr[i][key]
                                
                elif k == 'daily':
                    d = data[k]['data'][0]
                    time_int = int(day)
                    for key in d:
                        if time_int not in daily_data.keys():
                            print(time_int)
                            pass
                        else:
                            daily_data[time_int][sloc][key] = d[key]
        
#all_days_hourly_arrays[day] = allhours_outvector_gridlocs
with open(outdir + "daily_weather_dict.pck", 'wb') as f:
    pickle.dump(daily_data, f)
        
#all_days_hourly_arrays[day] = allhours_outvector_gridlocs
with open(outdir + "hourly_weather_dict.pck", 'wb') as f:
    pickle.dump(hourly_data, f)
                    
    

NameError: name 'daily_data' is not defined

In [61]:
#pick up where left off
#all_days_hourly_arrays[day] = allhours_outvector_gridlocs
with open(outdir + "daily_weather_dict.pck", 'rb') as f:
    daily_data = pickle.load(f)
        
#all_days_hourly_arrays[day] = allhours_outvector_gridlocs
with open(outdir + "hourly_weather_dict.pck", 'rb') as f:
    hourly_data = pickle.load(f)


In [10]:
with open(outdir + "data_keys.pck", 'rb') as f:
    lkeymap = pickle.load(f)


In [62]:
lkeymax

50

In [66]:
numdatapoints = lkeymax + 1

data_key_values = {}

for hour in hourly_data.keys():
    data_arrays = {}
    t = datetime.datetime.fromtimestamp(hour)
    year = str(t.year)
    write_path = outdir + "yearly/" + year
    if not os.path.exists(write_path):
        os.mkdir(write_path)
    dayt = datetime.datetime(year=t.year, month=t.month, day=t.day, hour=0, minute=0, second=0)
    dayint = int(time.mktime(dayt.timetuple()))
    #get the day and corresponding day data
    for l in grid_locs:
        tokens = l.split(",")
        lat = float(tokens[0])
        lon = float(tokens[1])
        data_arrays[l] = [ np.nan for i in range(numdatapoints) ]
        for k in hourly_data[hour][l].keys():
            data_arrays[l][lkeymap['hourly'][k]] = hourly_data[hour][l][k]
            if year < "2017": #getting min/max normalization values for training data only
                if k not in data_key_values.keys():
                    data_key_values[k] = set( [hourly_data[hour][l][k]] )
                else:
                    data_key_values[k].add(hourly_data[hour][l][k]) 
        for k in daily_data[dayint][l].keys():
            data_arrays[l][lkeymap['daily'][k]] = daily_data[dayint][l][k]
            if year < "2017":
                if k not in data_key_values.keys():
                    data_key_values[k] = set( [daily_data[dayint][l][k]] )
                else:
                    data_key_values[k].add(daily_data[dayint][l][k])

    
    with open(write_path + "/" + str(hour) + "_arrays.pck", 'wb') as d:
        pickle.dump(data_arrays, d)
        


In [21]:
#read in all written data arrays, find out what keys have nan values

all_data_arrays = {}

for year in years:
    hourfs = os.listdir(outdir + "yearly/" + str(year))
    for fname in hourfs:
        pth = outdir + "yearly/" + str(year) + "/" + fname
        with open(pth, 'rb') as d:
            data = pickle.load(d)
            all_data_arrays[fname] = data

In [48]:
nankeys = {}
nankeys['daily'] = {}
nankeys['hourly'] = {}
for f in all_data_arrays.keys():
    for l in grid_locs:
        for i in range(len(all_data_arrays[f][l])):
            val = all_data_arrays[f][l][i]
            if type(val) == str:
                pass
            elif np.isnan(val):
                for k in lkeymap.keys():
                    if k == 'daily' or k == 'hourly':
                        for lk in lkeymap[k].keys():
                            if lkeymap[k][lk] == i:
                                if lk not in nankeys[k].keys():
                                    nankeys[k][lk] = 1
                                else:
                                    nankeys[k][lk] += 1

In [44]:
lkeymap['hourly']

{'apparentTemperature': 46,
 'cloudCover': 50,
 'dewPoint': 47,
 'humidity': 42,
 'icon': 48,
 'precipAccumulation': 44,
 'precipIntensity': 43,
 'precipProbability': 37,
 'precipType': 35,
 'pressure': 36,
 'summary': 39,
 'temperature': 45,
 'time': 49,
 'visibility': 40,
 'windBearing': 38,
 'windSpeed': 41}

In [47]:
nankeys

{'daily': {}, 'hourly': {'humidity': 1}}

In [49]:
for k in nankeys:
    if k != 'latitude' or k != 'longitudue':
        for lk in nankeys[k]:
            print(lk, nankeys[k][lk]/(len(all_data_arrays) * 36))
            
#if the value is <1%, set to random unif(0,1)
    #if > 1, 
    #if time, set to -1
#if the value 
#humidity: set to 0 #value that would otherwise be 0
#visibility set to 1 #value that would otherwise be 1
#windbearing: set to random unif(0,1)
#summary: throw it out
#if string, set to -1
#precipAccumuluation: set to 0 #value that would otherwise be 0
#dewPoint: set to 0
#windSpeed: set to 0
#precipIntensity: set to 0
#cloudCover: set to 0
#pressure: throw it out



#pressure: throw it out
#temperatureHighTime: set to -1
#humidity: set to 0 or throw it out
#precipIntensity: set to -1
#icon set to -1
#apparentTemperatureMax: throw it out
#precipIntensityMax: -1
#temperatureHigh: 
#apparentTemperatureMaxTime: -1 or throw out
#precipProbability:  set to 0
#visilibity: set to 1
#windBearing: set to -1
#apparentTemperatureHighTime: set to -1
#temperatureMaxTime: random value in 0,1 or set to -1
#apparent

#need to interpolate values instead

#throw out
#pressure
#humidity
#precipType


pressure 0.616576579023613
temperatureHighTime 0.04945132551064754
humidity 0.981412067217152
precipIntensity 0.04945132551064754
icon 0.04945132551064754
apparentTemperatureMax 0.6048090504128639
precipIntensityMax 0.011189428509343763
temperatureHigh 0.011189428509343763
apparentTemperatureMaxTime 0.02670668549905838
precipProbability 0.026897725626539186
visibility 0.011949967405475881
windBearing 0.011949967405475881
apparentTemperatureHighTime 0.012688776618861364
temperatureMaxTime 2.17296827466319e-05
apparentTemperatureMinTime 4.34593654932638e-05
dewPoint 2.17296827466319e-05
cloudCover 4.34593654932638e-05
temperatureMinTime 2.17296827466319e-05
temperatureMin 2.17296827466319e-05
summary 2.17296827466319e-05
sunriseTime 0.0001955671447196871
time 2.17296827466319e-05
apparentTemperatureMin 0.0001955671447196871
precipIntensityMaxTime 2.17296827466319e-05
apparentTemperatureHigh 2.17296827466319e-05
apparentTemperatureLowTime 0.0001955671447196871
temperatureLow 4.34593654932

In [26]:
for k in data_key_values:
    vals = list(data_key_values[k])
    if type(vals[0]) == str:
        print(k)

summary
icon
precipType


In [27]:
data_key_values['icon'] #normalize to 0-1

{'clear-day',
 'clear-night',
 'cloudy',
 'fog',
 'partly-cloudy-day',
 'partly-cloudy-night',
 'rain',
 'snow',
 'wind'}

In [28]:
data_key_values['precipType'] #normalize to 0-1

{'rain', 'snow'}

In [5]:
36 * 8760 * 7

2207520

In [44]:
grid_locs
minlat = 25.8468651
maxlat = 36.5096362
minlon = -101.4818115
maxlon = -93.7463379

['25.8468651,-101.4818115',
 '25.8468651,-104.0603027',
 '25.8468651,-106.6387939',
 '25.8468651,-93.7463379',
 '25.8468651,-96.3248291',
 '25.8468651,-98.9033203',
 '27.97941932,-101.4818115',
 '27.97941932,-104.0603027',
 '27.97941932,-106.6387939',
 '27.97941932,-93.7463379',
 '27.97941932,-96.3248291',
 '27.97941932,-98.9033203',
 '30.11197354,-101.4818115',
 '30.11197354,-104.0603027',
 '30.11197354,-106.6387939',
 '30.11197354,-93.7463379',
 '30.11197354,-96.3248291',
 '30.11197354,-98.9033203',
 '32.24452776,-101.4818115',
 '32.24452776,-104.0603027',
 '32.24452776,-106.6387939',
 '32.24452776,-93.7463379',
 '32.24452776,-96.3248291',
 '32.24452776,-98.9033203',
 '34.37708198,-101.4818115',
 '34.37708198,-104.0603027',
 '34.37708198,-106.6387939',
 '34.37708198,-93.7463379',
 '34.37708198,-96.3248291',
 '34.37708198,-98.9033203',
 '36.5096362,-101.4818115',
 '36.5096362,-104.0603027',
 '36.5096362,-106.6387939',
 '36.5096362,-93.7463379',
 '36.5096362,-96.3248291',
 '36.5096362,

In [42]:
for k in data_key_values:
    vals = list(data_key_values[k])
    print(k, min(vals), max(vals)) #moon phase is already normalized, how nice
    
for l in grid_locs:
    tokens = grid_locs.split(",")
    minlat, maxlat
    minlon, maxlon

time 1293861600 1483254000
summary Breezy Windy until evening.
icon clear-day wind
precipType rain snow
temperature -18.04 110.76
apparentTemperature -43.45 137.14
dewPoint -22.63 91
humidity 0.02 1
windSpeed 0 114
windBearing 0 359
cloudCover 0 1
visibility 0 10
sunriseTime 1293886982 1483193980
sunsetTime 1293923475 1483230460
moonPhase 0 0.99
temperatureHigh 1.17 110.76
temperatureHighTime 1293886800 1483225200
temperatureLow -18.04 88
temperatureLowTime 1293940800 1483282800
apparentTemperatureHigh -8.53 137.14
apparentTemperatureHighTime 1293886800 1483225200
apparentTemperatureLow -43.45 100.22
apparentTemperatureLowTime 1293940800 1483279200
pressure 986.92 1051.23
temperatureMin -18.04 86
temperatureMinTime 1293879600 1483250400
temperatureMax 4.95 110.76
temperatureMaxTime 1293861600 1483225200
apparentTemperatureMin -43.45 99.8
apparentTemperatureMinTime 1293876000 1483250400
apparentTemperatureMax -8.49 137.14
apparentTemperatureMaxTime 1293861600 1483225200
precipIntensity 

In [31]:
valid_data_keys = []
for k in data_key_values:
    if k != "summary" or k != "time":
        valid_data_keys.append(k)

In [80]:
np.isnan(np.nan)

True

In [69]:
def norm_to_time_of_day(unixtime):
    if np.isnan(unixtime):
        val = np.nan
    else:
        dt = datetime.datetime.fromtimestamp(unixtime)
        h = dt.hour * 3600 + dt.minute * 60 + dt.second 
        val = h/86400
    return(val)

def norm_lat_long(val, latorlon):
    minlat = 25.8468651
    maxlat = 36.5096362
    minlon = -101.4818115
    maxlon = -93.7463379
    if latorlon == "latitude":
        out = (val - minlat)/(maxlat - minlat)
    elif latorlon == "longitude":
        out = (val - minlon)/(maxlon - minlon)
    
    return(out)

def str_to_norm_func(strin, key):
    if type(strin) == str:
        vals = sorted(list(data_key_values[key]))
        i = vals.index(strin)
        denom = len(vals) - 1
        out = i/denom
    else:
        out = np.nan
    return(out)

In [ ]:
#if the value is <1%, set to random unif(0,1)
    #if > 1, 
    #if time, set to -1
#if the value 
#humidity: set to 0 #value that would otherwise be 0
#visibility set to 1 #value that would otherwise be 1
#windbearing: set to random unif(0,1)
#summary: throw it out
#if string, set to -1
#precipAccumuluation: set to 0 #value that would otherwise be 0
#dewPoint: set to 0
#windSpeed: set to 0
#precipIntensity: set to 0
#cloudCover: set to 0
#pressure: throw it out


#set_nan_to_rand = all the rest
#set_nan_to_minus_one = string, time

In [86]:
#normalize sunriseTime and sunsetTime to time of day
#normalize temperatureHighTime to time of day
#normalize temperatureLowTime to time of day
#temperatureMinTime
#temperatureMaxTime
#apparentTemperatureMinTime
#apparentTemperatureMaxTime
#precipIntensityMaxTime
#normalize time to time of day

time_keys_to_norm = ['sunriseTime', 'sunsetTime', 'temperatureHighTime', 'temperatureLowTime', 'temperatureMinTime', 'temperatureMaxTime', 'apparentTemperatureMinTime', 'apparentTemperatureMaxTime', 'precipIntensityMaxTime', 'time']
values_to_norm = ['temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed', 'windBearing', 'visibility', 'temperatureHigh', 'temperatureLow', 'apparentTemperatureHigh', 'apparentTemperatureLow', 'pressure', 'temperatureMin', 'temperatureMax', 'apparentTemperatureMax', 'apparentTemperatureMin', 'precipIntensity', 'precipIntensityMax', 'precipAccumulation', 'latitude', 'longitude']
str_to_norm = ['icon', 'precipType']
values_no_norm = ['cloudCover', 'moonPhase', 'precipProbability']

throw_out = ['pressure', 'humidity', 'precipType']

minmaxs = {}
for k in values_to_norm:
    if k == "latitude" or k == "longitude":
        pass
    else:
        minmaxs[k] = {}
        vals = list(data_key_values[k])
        minmaxs[k]["max"] = max(vals)
        minmaxs[k]["min"] = min(vals)

In [21]:
with open(outdir + "training_minmaxs.pck", 'wb') as f:
    pickle.dump(minmaxs, f)

In [ ]:
with open(outdir + "training_minmaxs.pck", 'rb') as f:
    minmaxs = pickle.load(f)

In [88]:
for year in years:
    hourfs = os.listdir(outdir + "yearly/" + str(year))
    for fname in hourfs:
        pth = outdir + "yearly/" + str(year) + "/" + fname
        vec = []
        with open(pth, 'rb') as d:
            data = pickle.load(d)
            for l in grid_locs:
                gi = grid_locs.index(l)
                catvec = [ 0.0 for i in range(lkeymax + 1) ]
                for k in lkeymap.keys():
                    if k == "daily" or k == "hourly":
                        for lk in lkeymap[k].keys():
                            if lk in throw_out:
                                pass
                            else:
                                ind = lkeymap[k][lk]
                                dval = data[l][ind]
                                if lk in values_to_norm:
                                    catvec[ind] = (dval - minmaxs[lk]["min"])/(minmaxs[lk]["max"] - minmaxs[lk]["min"])
                                elif lk in str_to_norm:
                                    catvec[ind] = str_to_norm_func(dval, lk)
                                elif lk in values_no_norm:
                                    catvec[ind] = dval
                                elif lk in time_keys_to_norm:
                                    catvec[ind] = norm_to_time_of_day(dval)
                    elif k == "latitude" or k == "longitude":
                        pass
                vec += catvec
                                                                           
        vec = np.asarray(vec, dtype=np.float32)
        write_path = outdir + "yearly_normed_arrays/" + str(year)
        if not os.path.exists(write_path):
            os.mkdir(write_path)
        np.savetxt(write_path + "/" + fname.split("_")[0] + "_normed_vec.txt", vec, delimiter=",")
                        
                        
                     
                        
                    

In [90]:
all_normed_data_arrays = {}

for year in years:
    hourfs = os.listdir(outdir + "yearly_normed_arrays/" + str(year))
    for fname in hourfs:
        pth = outdir + "yearly_normed_arrays/" + str(year) + "/" + fname.split("_")[0] + "_normed_vec.txt"
        data = np.loadtxt(pth, delimiter=",")
        all_normed_data_arrays[fname] = data

In [97]:
hourfiles = list(all_normed_data_arrays.keys())
for i in range(len(hourfiles)):
    hf = all_normed_data_arrays[hourfiles[i]]
    if i % 10000 == 0:
        print(i)
    for j in range(len(hf)):
        val = hf[j] 
        if np.isnan(val):
            #average across all geographic locations
            avals = []
            for k in range(len(grid_locs)):
                locadd = int(-1.0*k*lkeymax)
                prv = [all_normed_data_arrays[hourfiles[i-3]][j + locadd], all_normed_data_arrays[hourfiles[i-2]][j + locadd], all_normed_data_arrays[hourfiles[i-1]][j + locadd]]
                nxt = [all_normed_data_arrays[hourfiles[i+1]][j + locadd], all_normed_data_arrays[hourfiles[i+2]][j + locadd], all_normed_data_arrays[hourfiles[i+3]][j + locadd]]
                a = prv + nxt
                avals.append(np.nanmean(np.array(a)))
            interp = np.nanmean(np.array(avals))
            if np.isnan(interp):
                print(hour)
                interp = -1.0
            all_normed_data_arrays[hourfiles[i]][j] = interp
    hour = int(hourfiles[i].split("_")[0])
    t = datetime.datetime.fromtimestamp(hour)
    year = str(t.year)
    if np.isnan(all_normed_data_arrays[hourfiles[i]]).any():
        print(hour)
    np.savetxt(outdir + "yearly_normed_arrays_clean/" + year + "/" + str(hour) + "_no_nans_interp_normed_vec.txt", all_normed_data_arrays[hourfiles[i]], delimiter=",")
    
    


0
10000
20000
30000
40000
50000
60000


IndexError: list index out of range

In [100]:
#fixing lag for last three
for i in range(len(hourfiles)-4,len(hourfiles)):
    hf = all_normed_data_arrays[hourfiles[i]]
    if i % 10000 == 0:
        print(i)
    for j in range(len(hf)):
        val = hf[j] 
        if np.isnan(val):
            #average across all geographic locations
            avals = []
            for k in range(len(grid_locs)):
                locadd = int(-1.0*k*lkeymax)
                prv = [all_normed_data_arrays[hourfiles[i-3]][j + locadd], all_normed_data_arrays[hourfiles[i-2]][j + locadd], all_normed_data_arrays[hourfiles[i-1]][j + locadd]]
                #nxt = [all_normed_data_arrays[hourfiles[i+1]][j + locadd], all_normed_data_arrays[hourfiles[i+2]][j + locadd], all_normed_data_arrays[hourfiles[i+3]][j + locadd]]
                a = prv #+ nxt
                avals.append(np.nanmean(np.array(a)))
            interp = np.nanmean(np.array(avals))
            if np.isnan(interp):
                print(hour)
                interp = -1.0
            all_normed_data_arrays[hourfiles[i]][j] = interp
    hour = int(hourfiles[i].split("_")[0])
    t = datetime.datetime.fromtimestamp(hour)
    year = str(t.year)
    if np.isnan(all_normed_data_arrays[hourfiles[i]]).any():
        print(hour)
    np.savetxt(outdir + "yearly_normed_arrays_clean/" + year + "/" + str(hour) + "_no_nans_interp_normed_vec.txt", all_normed_data_arrays[hourfiles[i]], delimiter=",")

In [15]:
lkeymax = 0
for k in lkeymap:
    if lkeymap[k] > lkeymax:
        lkeymax = lkeymap[k]

TypeError: '>' not supported between instances of 'dict' and 'int'

In [14]:
#now I want to do some feature engineering with the weather data
#reset the notebook and load arrays
all_normed_data_arrays = {}

max_over_bill = sorted([14, 35, 10, 24, 11, 47])
min_over_bill = sorted([33, 23, 35, 25, 47])
#total_over_bill = [16, 46]
average_over_two_weeks = sorted([24, 25, 11, 32, 22, 34, 16, 6, 20, 17, 10, 33, 14, 23, 18, 47, 49, 48, 46, 43, 45, 35, 36])

single_loc_keys = {}
j = 0
for i in max_over_bill + min_over_bill + average_over_two_weeks:
    single_loc_keys[i] = j
    j += 1
single_loc = [ 0.0 for s in range(len(max_over_bill + min_over_bill + average_over_two_weeks))]

for year in years:
    hourfs = os.listdir(outdir + "yearly_normed_arrays_clean/" + str(year))
    for fname in hourfs:
        pth = outdir + "yearly_normed_arrays_clean/" + str(year) + "/" + fname.split("_")[0] + "_no_nans_interp_normed_vec.txt"
        data = np.loadtxt(pth, delimiter=",")
        all_normed_data_arrays[fname] = data

hourfiles = sorted(list(all_normed_data_arrays.keys()))
all_arrays = []

for i in range(len(hourfiles)):
    all_arrays.append(all_normed_data_arrays[hourfiles[i]])

In [8]:
curr_year = 2011
for i in range(len(all_arrays)):
    ut = hourfiles[i].split("_")[0]
    year = datetime.datetime.fromtimestamp(ut).year
    maxob = [0.0 for i in range(len(max_over_bill))]
    minob = [100.0 for i in range(len(min_over_bill))]
    if year > curr_year:
        maxob = [0.0 for i in range(len(max_over_bill))]
        minob = [100.0 for i in range(len(min_over_bill))]
    for k in range(len(grid_locs)):
        catvec = copy.copy(single_loc)
        for j in range(lkeymax): #all features
            ind = k*lkeymax + j
            if j in max_over_bill: #going to need to catch leap years
                catind = single_loc_keys[j]
                val = all_arrays[i][ind]
                running_val = maxob[max_over_bill.index(j)]
                if val > running_val:
                    maxob[maxob.index(j)] = val
                    running_val = val
                catvec[catind] = running_val
            if j in min_over_bill:
                catind = single_loc_keys[j]
                val = all_arrays[i][ind]
                running_val = maxob[maxob.index(j)]
                if val < running_val:
                    maxob[min_over_bill.index(j)] = val
                    running_val = val
                catvec[catit] = running_val
            if j in average_over_two_weeks:
                catind = single_loc_keys[j]
                vals = []
                #look at the last 168 values at the same index
                for t in range(168):
                    vals.append(all_arrays[i - t][ind])
                val = np.nanmean(vals)
                catvec[catind] = val
    out = np.concatenate((all_arrays[i], catvec), axis=0)
    hour = hourfiles[i].split("_")[0]
    np.savetxt(outdir + "yearly_normed_arrays_clean/" + year + "/" + hour + "_clean_normed_feature_vec.txt", out, delimiter=",")
    

IndentationError: expected an indented block (<ipython-input-8-90a20e974fc7>, line 33)

In [16]:
all_arrays[0]

array([0.92000002, 0.        , 0.83333331, ..., 0.        , 0.13      ,
       0.        ])

In [ ]:
for i in range(len(all_arrays)):
    for k in range(len(grid_locs)):
        for j in range(50):

In [11]:
lkeymax

NameError: name 'lkeymax' is not defined

In [ ]:
{'daily': {'apparentTemperatureHigh': 24,
  'apparentTemperatureHighTime': 7,
  'apparentTemperatureLow': 25,
  'apparentTemperatureLowTime': 1,
  'apparentTemperatureMax': 11,
  'apparentTemperatureMaxTime': 31,
  'apparentTemperatureMin': 32,
  'apparentTemperatureMinTime': 2,
  'cloudCover': 22,
  'dewPoint': 15,
  'humidity': 34,
  'icon': 21,
  'moonPhase': 0,
  'precipAccumulation': 16,
  'precipIntensity': 6,
  'precipIntensityMax': 20,
  'precipIntensityMaxTime': 19,
  'precipProbability': 17,
  'precipType': 28,
  'pressure': 3,
  'summary': 4,
  'sunriseTime': 29,
  'sunsetTime': 5,
  'temperatureHigh': 10,
  'temperatureHighTime': 13,
  'temperatureLow': 33,
  'temperatureLowTime': 8,
  'temperatureMax': 14,
  'temperatureMaxTime': 27,
  'temperatureMin': 23,
  'temperatureMinTime': 26,
  'time': 9,
  'visibility': 30,
  'windBearing': 12,
  'windSpeed': 18},
 'hourly': {'apparentTemperature': 47,
  'cloudCover': 49,
  'dewPoint': 41,
  'humidity': 48,
  'icon': 42,
  'precipAccumulation': 46,
  'precipIntensity': 43,
  'precipProbability': 45,
  'precipType': 40,
  'pressure': 38,
  'summary': 39,
  'temperature': 35,
  'time': 50,
  'visibility': 44,
  'windBearing': 37,
  'windSpeed': 36}}